In [ ]:
#url_list 만들기

import datetime       

start_day = datetime.datetime(2015,12,1)   
one_day = datetime.timedelta(days = 1)    
end_day = datetime.datetime(2016,1,1)       

url_list = []
while start_day < end_day :
    dtdt = start_day.strftime("%Y%m%d")
    url_list.append("https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=1_1&area=00&begin_date={}".format(dtdt))    
    start_day = start_day + one_day

In [ ]:
#크롤링해서 JSON에 저장(문제는 중복된 프로그램명은 낮은 시청률을 반영한다는 것, 보수적 접근)

import urllib.request
from bs4 import BeautifulSoup
from collections import OrderedDict
import json

pn_list = []

for base_url in url_list:
    pnss = OrderedDict()
    with urllib.request.urlopen(base_url) as url:
    
        doc = url.read()
        soup = BeautifulSoup(doc, "html.parser")
        names = soup.find_all("td", class_="tb_txt")
        percents10 = soup.find_all("td", class_="percent")
        percents20 = soup.find_all("td", class_="percent_g")
    
        name_list = []
        percent_list = []
        pnss["날짜"] = base_url[-8:]
        for name in names[0:20]:
            name_list.append(name.text.strip())
        for percent10 in percents10[0:10]:
            percent_list.append(percent10.text.strip())
        for percent20 in percents20[0:10]:
            percent_list.append(percent20.text.strip())
        for i in range(0,20):
            pnss[name_list[i]] = percent_list[i]
        

    pn_list.append(pnss)
    
with open('rating2015_m12.json', 'w', encoding="utf-8") as make_file:
    json.dump(pn_list, make_file, ensure_ascii=False)

In [ ]:
#JSON을 불러와서 시청률평균 구해보기

import json
import numpy as np

p_name_list = ['무한도전', "황금어장라디오스타", "일밤(미스터리음악쇼복면가왕,리얼입대프로젝트진짜사나이2)", "마이리틀텔레비전"]

json_data = []
with open("rating2015_m12.json", encoding = "UTF-8") as file:
    data = file.readlines()
    for d in data:
        json_data.append(json.loads(d))
        
for p_name in p_name_list: 
    p_rating = []
    for i in range(0,len(json_data[0])):
        if p_name in json_data[0][i].keys():
            p_rating.append(float(json_data[0][i][p_name]))
    print(p_name, np.mean(p_rating))